<a href="https://colab.research.google.com/github/AuroraLin27/AuroraLin27.github.io/blob/master/Mask_Detection_skeleton_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **0. Mask augmentation**

*   mount google drive
*   MaskTheFace github (https://github.com/aqeelanwar/MaskTheFace.git)
*   include mask augmented face image samples in report



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd '/content/drive/MyDrive/detect_mask'

In [ ]:
!unzip data.zip

In [ ]:
! git clone https://github.com/aqeelanwar/MaskTheFace.git

In [ ]:
! pip install scikit-learn
! pip install wandb
! pip install dlib
! pip install face-recognition
! pip install face-recognition-models
! pip install dotmap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 66.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.5/289.5 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566170 sha256=886eb0cb4e6dea3f1a7505aa3a75465d8d29884638f8c04040a2399960f7b31b
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models


In [ ]:
cd 'MaskTheFace'

In [ ]:
# print current directory
import os
os.getcwd()

In [ ]:
! python mask_the_face.py --path "/content/drive/MyDrive/detect_mask/data/train/not_wearing_mask" --mask_type "random"

In [ ]:
! python mask_the_face.py --path "/content/drive/MyDrive/detect_mask/data/val/not_wearing_mask" --mask_type "random"

# **1. Prepare Data for Training**

*   data_loader using *torchvision.datasets.ImageFolder* for Custom dataset
*   **image augmentation** in *transforms*
*   include augmented face image samples in report



In [ ]:
import torch
import torchvision
import os
from torchvision import transforms,datasets
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder


In [ ]:
# Set the dataset path
train_path = "/content/drive/MyDrive/data/train"
val_path = "/content/drive/MyDrive/data/val"


# Define the transformation operations for training data
train_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),  # Random horizontal flip
    transforms.RandomVerticalFlip(),    # Random vertical flip
    transforms.CenterCrop(112),         # Center crop to 112x112
    transforms.ToTensor()               # Convert image to tensor and normalize pixel values to [0, 1]
])

# Define the transformation operations for validation data
val_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),  # Random horizontal flip
    transforms.RandomVerticalFlip(),    # Random vertical flip
    transforms.CenterCrop(112),         # Center crop to 112x112
    transforms.ToTensor()               # Convert image to tensor and normalize pixel values to [0, 1]
])

#Check for any hidden files or directories within val_path and remove them
!ls -a {val_path} #List all files including hidden files
!rm -rf {val_path}/.ipynb_checkpoints #Remove .ipynb_checkpoint directory if it exists

# Create datasets
train_dataset = ImageFolder(root=train_path, transform=train_transforms)
val_dataset = ImageFolder(root=val_path, transform=val_transforms)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)


not_wearing_mask  wearing_mask


In [ ]:
def check_loader(loader):
    for inputs, labels in loader:
        print(f'Inputs dtype: {inputs.dtype}, shape: {inputs.shape}')
        print(f'Labels dtype: {labels.dtype}, shape: {labels.shape}')
        print(f'Labels range: {labels.min()} - {labels.max()}')
        break

check_loader(train_loader)
check_loader(val_loader)

Inputs dtype: torch.float32, shape: torch.Size([64, 3, 112, 112])
Labels dtype: torch.int64, shape: torch.Size([64])
Labels range: 0 - 1
Inputs dtype: torch.float32, shape: torch.Size([64, 3, 112, 112])
Labels dtype: torch.int64, shape: torch.Size([64])
Labels range: 0 - 0


# **2. Prepare Model**

*   Pytorch ResNet - *ref*. https://github.com/pytorch/vision/blob/main/torchvision/models/resnet.py
*   use **ResNet50** from torchvision.model_zoo
*   explore more models in https://pytorch.org/vision/stable/models.html
*   **change the dimension of the classifier**

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [ ]:
# assign device cpu or gpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
# import packages for importing models
import torchvision.models as models
import torch.nn as nn


In [ ]:
# Load the pretrained ResNet-50 model
try:
    model = models.resnet50(pretrained=False)
except Exception as e:
    print(f"Error loading model: {e}")


# Modify the last fully connected layer to output 1 class
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 1)


# Move the model to the device
try:
    model = model.to(device)
except Exception as e:
    print(f"Error moving model to device: {e}")


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


# **3. Training**


*   write **training code** including belows:
   - hyper parameters such as batch size, learning rate, epoch
   - criterion(loss function such as BCELoss), optimizer(eg. Adam, SGD, etc.)  and scheduler
   - save model weight

*   **print training/validation loss and accuracy** per epoch or iteration
*   inlcude visualizer, **tensorboard**, to show training/validation accuracy and loss


In [ ]:
# hyperparameters
batch_size = 64
learning_rate = 0.01
num_epochs=50

In [ ]:
import torch
import wandb
import sklearn
from sklearn import metrics
import datetime
import os
import torch.optim as optim
import torchvision.models as models
import matplotlib.pyplot as plt
wandb.login()
# use wandb.init
# initialized WandB
wandb.init(project="mask-detection", config={
    "learning_rate": learning_rate,
    "epochs": num_epochs,
    "batch_size": batch_size
})

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [ ]:
# Define the loss function
criterion = nn.BCEWithLogitsLoss()

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Define the learning rate scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)


In [ ]:
def train_model():
    model.train()  # Set the model to training mode
    running_loss = 0.0
    corrects = 0

    for inputs, labels in train_loader:
        # Check data types and ranges
        assert inputs.dtype == torch.float32, f"Expected inputs of type torch.float32 but got {inputs.dtype}"

        # Convert labels to float type
        labels = labels.float()
        assert labels.dtype == torch.float32, f"Expected labels of type torch.float32 but got {labels.dtype}"

        try:
            inputs = inputs.to(device)
            labels = labels.unsqueeze(1).to(device)  # Expand labels to match the shape of model outputs
        except Exception as e:
            print(f"Error moving data to device: {e}")
            continue

        optimizer.zero_grad()

        with torch.set_grad_enabled(True):
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)  # Get predictions
            loss = criterion(outputs, labels)  # Calculate the loss

            loss.backward()  # Backpropagation
            optimizer.step()  # Update model parameters

        running_loss += loss.item() * inputs.size(0)
        corrects += torch.sum(preds == labels.data).item()

    epoch_loss = running_loss / len(train_loader.dataset)  # Calculate epoch loss
    epoch_acc = corrects / len(train_loader.dataset)  # Calculate epoch accuracy
    return epoch_loss, epoch_acc


In [ ]:
def val_model():
    model.eval()  # Set the model to evaluation mode
    running_loss = 0.0
    corrects = 0
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for inputs, labels in val_loader:
            # Check data types and ranges
            assert inputs.dtype == torch.float32, f"Expected inputs of type torch.float32 but got {inputs.dtype}"

            # Convert labels to float type to match the loss function's expectation
            labels = labels.float()

            inputs = inputs.to(device)
            labels = labels.unsqueeze(1).to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)  # Get predictions
            corrects += torch.sum(preds == labels.data).item()

            all_labels.extend(labels.cpu().numpy())
            # Extract probabilities for the positive class
            all_preds.extend(torch.sigmoid(outputs).cpu().numpy().squeeze())  # Use sigmoid and squeeze for binary classification

    epoch_loss = running_loss / len(val_loader.dataset)  # Calculate epoch loss
    epoch_acc = corrects / len(val_loader.dataset)  # Calculate epoch accuracy

    # Calculate ROC and AUC
    fpr, tpr, _ = metrics.roc_curve(all_labels, all_preds)
    roc_auc = metrics.auc(fpr, tpr)

    return epoch_loss, epoch_acc, fpr, tpr, roc_auc


In [ ]:
for epoch in range(num_epochs):
    # Perform training
    train_loss, train_acc = train_model()

    # Perform validation
    val_loss, val_acc, fpr, tpr, roc_auc = val_model()

    # Log metrics to WandB
    wandb.log({
        "Epoch": epoch + 1,
        "Train Loss": train_loss,
        "Train Accuracy": train_acc,
        "Validation Loss": val_loss,
        "Validation Accuracy": val_acc,
        "ROC AUC": roc_auc
    })

    # Print training and validation results for the current epoch
    print(f'Epoch {epoch+1}/{num_epochs}')
    print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}')
    print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, ROC AUC: {roc_auc:.4f}')

    # Update the learning rate
    scheduler.step()

    # Save the model weights
    torch.save(model.state_dict(), '/content/drive/MyDrive/detect_mask/model.pth')

print('The model has been trained and saved.')

# Perform validation and print results
val_loss, val_acc, fpr, tpr, roc_auc = val_model()
print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, ROC AUC: {roc_auc:.4f}')


KeyboardInterrupt: 